In [15]:
import pandas as pd

In [16]:
Tweet = pd.read_csv("stockerbot-export.csv")

In [17]:
Tweet

,id,text,timestamp,source,symbols,company_names,url,verified,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1019696670777503700,VIDEO: “I was in my office. I was minding my o...,Wed Jul 18 21:33:26 +0000 2018,GoldmanSachs,GS,The Goldman Sachs,https://twitter.com/i/web/status/1019696670777...,TRUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1019709091038548000,The price of lumber $LB_F is down 22% since hi...,Wed Jul 18 22:22:47 +0000 2018,StockTwits,M,Macy's,https://twitter.com/i/web/status/1019709091038...,TRUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1019711413798035500,Who says the American Dream is dead? https://t...,Wed Jul 18 22:32:01 +0000 2018,TheStreet,AIG,American,https://buff.ly/2L3kmc4,TRUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1019716662587740200,Barry Silbert is extremely optimistic on bitco...,Wed Jul 18 22:52:52 +0000 2018,MarketWatch,BTC,Bitcoin,https://twitter.com/i/web/status/1019716662587...,TRUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1019718460287389700,How satellites avoid attacks and space junk wh...,Wed Jul 18 23:00:01 +0000 2018,Forbes,ORCL,Oracle,http://on.forbes.com/6013DqDDU,TRUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28431,1019730074797408300,RT @cryptorangutang: To all the fallen heroes ...,Wed Jul 18 23:46:10 +0000 2018,_DaddyDanger,BTC,Bitcoin,NaN,FALSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28432,1019730082703790100,RT @ENewsMan: New exchange for Telcoin!?!?! Mi...,Wed Jul 18 23:46:12 +0000 2018,iad81,BTC,Bitcoin,NaN,FALSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28433,1019730088617635800,$FB : 29234a9c-7f08-4d5a-985f-cb1a5554ecf9,Wed Jul 18 23:46:13 +0000 2018,test5f1798,FB,Facebook,NaN,FALSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28434,1019730115805184000,RT @invest_in_hd: 'Nuff said! $TEL #telcoin #...,Wed Jul 18 23:46:19 +0000 2018,iad81,BTC,Bitcoin,https://twitter.com/CRYPTOVERLOAD/status/10178...,FALSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
tweets = Tweet['text'][0:50]

In [19]:
tweets1 = "".join(tweets)

In [29]:
!pip install python-dotenv
!pip install openai

from dotenv import load_dotenv
import os

load_dotenv()

from openai import AzureOpenAI  
combined_tweets = tweets1
endpoint = os.getenv('END_POINT') 
deployment = "gpt-35-turbo-2" 
subscription_key = os.getenv('API_KEY') 

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",  
)  
    
# Prepare the chat prompt  
chat_prompt = [
{
    "role": "system",
    "content": "You are an AI assistant that helps people find information."
}
,
 {"role": "user", "content": f"Analyse the following combined tweets: '{combined_tweets}' and identify the business segment (finance, technology, sports) and customer income profile (Low, Medium, High). Determine the overall sentiment of the tweets (Positive, Negative, Neutral) and output your answer in the following table format on each segment for :: {{BUSINESS_SEGMENT}} : {{INCOME_PROFILE}} : {{SENTIMENT_SCORE}} "}] 
   
speech_result = chat_prompt  

# Generate the completion  
completion = client.chat.completions.create(  
    model=deployment,  
    messages=speech_result,   
    max_tokens=800,
    temperature=0
)  

tweet_response = completion.choices[0].message.content.strip()
    
print(tweet_response)


| BUSINESS_SEGMENT | INCOME_PROFILE | SENTIMENT_SCORE |
|------------------|----------------|-----------------|
| Finance          | High           | Neutral         |
| Finance          | Medium         | Neutral         |
| Finance          | Low            | Negative        |
| Technology       | High           | Positive        |
| Technology       | Medium         | Neutral         |
| Technology       | Low            | Neutral         |
| Sports           | High           | Positive        |
| Sports           | Medium         | Neutral         |
| Sports           | Low            | Negative        |


In [21]:
import pandas as pd
import io
# Clean the tweet response to remove the table borders
cleaned_response = tweet_response.strip().split('\n')[2:]  # Skip the first two lines
cleaned_response = [line.strip().strip('|').split('|') for line in cleaned_response]  # Clean each line
 
# Create the DataFrame from the cleaned response
df = pd.DataFrame(cleaned_response, columns=['BUSINESS_SEGMENT', 'INCOME_PROFILE', 'SENTIMENT_SCORE'])
 
# Strip any extra whitespace from the DataFrame cells
tweet_data_df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
 
# Display the DataFrame
print(tweet_data_df)
# df = pd.DataFrame(data)
# print(df)


  BUSINESS_SEGMENT INCOME_PROFILE SENTIMENT_SCORE
0          Finance           High         Neutral
1          Finance         Medium         Neutral
2          Finance            Low        Negative
3       Technology           High        Positive
4       Technology         Medium         Neutral
5       Technology            Low         Neutral
6           Sports           High        Positive
7           Sports         Medium         Neutral
8           Sports            Low        Negative


C:\Users\NBaggi\AppData\Local\Temp\ipykernel_16404\2764201573.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tweet_data_df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [28]:
import numpy as np
import pandas as pd
# Increase the sample size to 10,000 entries for a more robust training dataset
num_samples = 10000

# Generate borrower data within the credit score range of 700-750
data_sample = {
    "Credit_Score": np.random.randint(700, 751, num_samples),
    "Annual_Income": np.random.randint(40000, 120001, num_samples),
    "Loan_Amount": np.random.randint(10000, 100001, num_samples),
    "LTV_Ratio": np.round(np.random.uniform(0.5, 0.9, num_samples), 2),
    "Credit_Utilization_Ratio": np.round(np.random.uniform(0.1, 0.8, num_samples), 2),
    "Missed_Payments_Past_Year": np.random.randint(0, 4, num_samples),
    "Avg_Monthly_Balance": np.random.randint(500, 10001, num_samples),
    "Employment_Status": np.random.choice([0, 1], num_samples, p=[0.1, 0.9]),
    "Recent_Large_Transactions": np.random.choice([0, 1], num_samples, p=[0.85, 0.15]),
    "Credit_History": np.random.choice([0, 1], num_samples, p=[0.6, 0.4]),
    "Savings_Account_Balance": np.random.randint(0, 100001, num_samples),
    "Credit_Card_Balance": np.random.randint(0, 100001, num_samples),
    "Industry": np.random.choice(['Technology', 'Sports','Finance'], num_samples),
    "Age": np.random.randint(21, 65, num_samples),
    "Education_Level": np.random.choice(['High School', 'Bachelor’s', 'Master’s', 'PhD'], num_samples),
    "Marital_Status": np.random.choice(['Single', 'Married', 'Divorced'], num_samples),
    "Number_of_Dependents": np.random.randint(0, 5, num_samples),
    "Total_Debt": np.random.randint(5000, 150001, num_samples),
    "Debt_to_Income_Ratio": np.round(np.random.uniform(0.1, 0.6, num_samples), 2),
    "Past_Delinquencies_2_5_Years": np.random.randint(0, 3, num_samples),
    "Years_in_Current_Job": np.random.randint(1, 20, num_samples),
}
# Simulate delinquency status based on a basic heuristic for illustrative purposes (Delinquency Status: Binary outcome, with 1 indicating a delinquent borrower and 0 indicating a non-delinquent borrower.)

# Define weights for each feature contributing to delinquency
"""
Positive Weights:
                Signify Risk-Increasing Factors: When a feature has a positive weight, it means that an increase in that feature's value contributes positively to the risk of delinquency.
                                                 For example, if Debt_to_Income_Ratio has a positive weight, higher debt relative to income is seen as increasing the likelihood of the customer being delinquent.
Negative Weights:
                Signify Risk-Reducing Factors: A negative weight implies that an increase in that feature's value contributes negatively to the risk of delinquency, reducing the likelihood of delinquency.
                                               For example, a higher Credit_Score is associated with a lower risk of delinquency, so it has a negative weight to indicate that as the credit score increases, the probability of delinquency decreases.
"""
weights = {
    "Credit_Score": -0.5,  # Lower credit scores increase delinquency risk
    "Debt_to_Income_Ratio": 0.4,  # Higher DTI increases risk
    "Missed_Payments_Past_Year": 0.5,  # More missed payments increase risk
    "Total_Debt": 0.3,  # Higher total debt increases risk
    "Employment_Status": -0.2,  # 1 means employed, reducing risk
    "Past_Delinquencies_2_5_Years": 0.4,  # Past delinquencies increase risk
    "Recent_Large_Transactions": 0.3,  # Indicates potential financial stress
    "Savings_Account_Balance": -0.3,  # Higher savings reduce risk
    "Credit_History": -0.3,  # Positive credit history reduces risk
    "Credit_Utilization_Ratio": 0.4,  # Higher utilization increases risk
    "Years_in_Current_Job": -0.2  # Longer job tenure reduces risk
}

# Apply the rule to calculate a risk score for each row
def calculate_delinquency_risk(row):
    risk_score = (
        weights["Credit_Score"] * (750 - row["Credit_Score"]) / 100 +  # Normalize score
        weights["Debt_to_Income_Ratio"] * row["Debt_to_Income_Ratio"] +
        weights["Missed_Payments_Past_Year"] * row["Missed_Payments_Past_Year"] +
        weights["Total_Debt"] * row["Total_Debt"] / 150000 +  # Normalize debt
        weights["Employment_Status"] * row["Employment_Status"] +
        weights["Past_Delinquencies_2_5_Years"] * row["Past_Delinquencies_2_5_Years"] +
        weights["Recent_Large_Transactions"] * row["Recent_Large_Transactions"] +
        weights["Savings_Account_Balance"] * (100000 - row["Savings_Account_Balance"]) / 100000 +  # Normalize savings
        weights["Credit_History"] * row["Credit_History"] +
        weights["Credit_Utilization_Ratio"] * row["Credit_Utilization_Ratio"] +
        weights["Years_in_Current_Job"] * (20 - row["Years_in_Current_Job"]) / 20  # Normalize job tenure
    )
    return 1 if risk_score > 1.5 else 0  # Threshold for classification
# Create a DataFrame
customer_data = pd.DataFrame(data_sample)

# Apply the function to the DataFrame to create the Delinquency column
customer_data["Delinquency"] = customer_data.apply(calculate_delinquency_risk, axis=1)

# Display the first few rows of the larger dataset
customer_data.head()

,Credit_Score,Annual_Income,Loan_Amount,LTV_Ratio,Credit_Utilization_Ratio,Missed_Payments_Past_Year,Avg_Monthly_Balance,Employment_Status,Recent_Large_Transactions,Credit_History,...,Industry,Age,Education_Level,Marital_Status,Number_of_Dependents,Total_Debt,Debt_to_Income_Ratio,Past_Delinquencies_2_5_Years,Years_in_Current_Job,Delinquency
0,719,93243,35116,0.87,0.47,2,4617,1,0,1,...,Finance,54,High School,Divorced,2,55631,0.27,1,8,0
1,740,118333,77788,0.80,0.62,1,6580,1,0,0,...,Finance,43,High School,Married,4,99836,0.32,1,14,0
2,739,41247,21274,0.68,0.53,3,2564,1,0,0,...,Sports,32,High School,Single,2,68478,0.14,0,16,0
3,749,114989,51745,0.63,0.73,2,3844,1,0,0,...,Finance,29,Bachelor’s,Single,1,37086,0.58,2,13,1
4,704,92102,29397,0.62,0.79,3,8794,1,0,0,...,Sports,28,PhD,Single,3,126794,0.17,0,5,0


In [23]:
#Cust data 
def get_income_profile(Annual_Income):
    if Annual_Income < 60000:
        return 'Low'
    elif 60000 <= Annual_Income <= 90000:
        return 'Medium'
    else:
        return 'High'
# Apply the function to create a new column 'Income_profile'
customer_data['Income_Profile'] = customer_data['Annual_Income'].apply(get_income_profile)
customer_data

 

,Credit_Score,Annual_Income,Loan_Amount,LTV_Ratio,Credit_Utilization_Ratio,Missed_Payments_Past_Year,Avg_Monthly_Balance,Employment_Status,Recent_Large_Transactions,Credit_History,...,Age,Education_Level,Marital_Status,Number_of_Dependents,Total_Debt,Debt_to_Income_Ratio,Past_Delinquencies_2_5_Years,Years_in_Current_Job,Delinquency,Income_Profile
0,736,72998,88448,0.76,0.45,2,4335,1,0,1,...,23,Bachelor’s,Divorced,1,111285,0.44,2,19,1,Medium
1,726,80946,90601,0.61,0.40,1,3363,1,0,0,...,45,Bachelor’s,Married,1,107551,0.59,1,17,0,Medium
2,708,64471,25821,0.64,0.74,1,3293,1,0,1,...,27,PhD,Divorced,3,55853,0.17,1,7,0,Medium
3,741,47142,51346,0.62,0.60,0,3368,1,0,0,...,47,Bachelor’s,Divorced,3,103929,0.27,0,10,0,Low
4,733,80335,70922,0.87,0.31,3,7833,1,0,1,...,43,PhD,Divorced,0,60090,0.41,2,6,1,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,737,117495,68421,0.62,0.66,3,9575,1,0,1,...,46,PhD,Married,2,7763,0.19,1,11,0,High
9996,704,56581,41163,0.74,0.58,0,7273,1,0,1,...,41,PhD,Divorced,0,134624,0.30,1,16,0,Low
9997,727,119201,25660,0.59,0.10,3,3696,1,1,0,...,22,Master’s,Married,0,69684,0.25,2,3,1,High
9998,702,115433,27948,0.88,0.48,3,9932,1,0,1,...,21,Bachelor’s,Single,2,13814,0.46,2,1,1,High


In [24]:
 
# Join the DataFrames on 'Industry' and 'Income_profile'/'Incomeprofile'
merged_df = pd.merge(customer_data, tweet_data_df, how='inner', left_on=['Industry', 'Income_Profile'], right_on=['BUSINESS_SEGMENT', 'INCOME_PROFILE'])

# Drop the redundant 'Incomeprofile' and 'business_segment' columns from the merged DataFrame
merged_df = merged_df.drop(columns=['INCOME_PROFILE', 'BUSINESS_SEGMENT'])

# Display the merged DataFrame
merged_df
customer_data_merged = merged_df


In [25]:
merged_df

,Credit_Score,Annual_Income,Loan_Amount,LTV_Ratio,Credit_Utilization_Ratio,Missed_Payments_Past_Year,Avg_Monthly_Balance,Employment_Status,Recent_Large_Transactions,Credit_History,...,Education_Level,Marital_Status,Number_of_Dependents,Total_Debt,Debt_to_Income_Ratio,Past_Delinquencies_2_5_Years,Years_in_Current_Job,Delinquency,Income_Profile,SENTIMENT_SCORE
0,736,72998,88448,0.76,0.45,2,4335,1,0,1,...,Bachelor’s,Divorced,1,111285,0.44,2,19,1,Medium,Neutral
1,726,80946,90601,0.61,0.40,1,3363,1,0,0,...,Bachelor’s,Married,1,107551,0.59,1,17,0,Medium,Neutral
2,708,64471,25821,0.64,0.74,1,3293,1,0,1,...,PhD,Divorced,3,55853,0.17,1,7,0,Medium,Neutral
3,741,47142,51346,0.62,0.60,0,3368,1,0,0,...,Bachelor’s,Divorced,3,103929,0.27,0,10,0,Low,Neutral
4,733,80335,70922,0.87,0.31,3,7833,1,0,1,...,PhD,Divorced,0,60090,0.41,2,6,1,Medium,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,737,117495,68421,0.62,0.66,3,9575,1,0,1,...,PhD,Married,2,7763,0.19,1,11,0,High,Positive
9996,704,56581,41163,0.74,0.58,0,7273,1,0,1,...,PhD,Divorced,0,134624,0.30,1,16,0,Low,Neutral
9997,727,119201,25660,0.59,0.10,3,3696,1,1,0,...,Master’s,Married,0,69684,0.25,2,3,1,High,Positive
9998,702,115433,27948,0.88,0.48,3,9932,1,0,1,...,Bachelor’s,Single,2,13814,0.46,2,1,1,High,Neutral


In [26]:
#Pre processing data
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

# Encode categorical variables
# For binary and simple categorical features, use LabelEncoder
le_industry = LabelEncoder()
le_education = LabelEncoder()
le_marital = LabelEncoder()
le_SENTIMENT_SCORE = LabelEncoder()
le_income_profile = LabelEncoder()
customer_data_merged['Industry'] = le_industry.fit_transform(customer_data_merged['Industry'])
customer_data_merged['Education_Level'] = le_education.fit_transform(customer_data_merged['Education_Level'])
customer_data_merged['Marital_Status'] = le_marital.fit_transform(customer_data_merged['Marital_Status'])
customer_data_merged['SENTIMENT_SCORE'] = le_SENTIMENT_SCORE.fit_transform(customer_data_merged['SENTIMENT_SCORE'])
customer_data_merged['Income_Profile'] = le_income_profile.fit_transform(customer_data_merged['Income_Profile'])

ModuleNotFoundError: No module named 'sklearn'

In [217]:
# Scale numerical features
scaler = StandardScaler()
numerical_features = [
    'Credit_Score', 'Annual_Income', 'Loan_Amount', 'LTV_Ratio',
    'Credit_Utilization_Ratio', 'Avg_Monthly_Balance', 'Savings_Account_Balance',
    'Credit_Card_Balance', 'Age', 'Total_Debt', 'Debt_to_Income_Ratio',
    'Years_in_Current_Job'
]
customer_data_merged[numerical_features] = scaler.fit_transform(customer_data_merged[numerical_features])

print(customer_data_merged.head())
customer_data_merged.head()

   Credit_Score  Annual_Income  Loan_Amount  LTV_Ratio  \
0      0.470195       0.912798     0.188360  -1.390131   
1      0.740538      -1.270699     1.044588  -0.613757   
2     -1.489795       0.127785     0.047786   0.248879   
3     -0.138078       0.176391     1.141145   0.421407   
4     -0.949108      -0.265368    -1.505382  -1.390131   

   Credit_Utilization_Ratio  Missed_Payments_Past_Year  Avg_Monthly_Balance  \
0                 -0.787119                          3            -1.029514   
1                 -0.045594                          3            -0.947514   
2                 -0.095029                          3             1.728081   
3                  0.597061                          2            -1.152513   
4                 -0.144464                          1             0.119210   

   Employment_Status  Recent_Large_Transactions  Credit_History  ...  \
0                  1                          0               1  ...   
1                  1            

,Credit_Score,Annual_Income,Loan_Amount,LTV_Ratio,Credit_Utilization_Ratio,Missed_Payments_Past_Year,Avg_Monthly_Balance,Employment_Status,Recent_Large_Transactions,Credit_History,...,Education_Level,Marital_Status,Number_of_Dependents,Total_Debt,Debt_to_Income_Ratio,Past_Delinquencies_2_5_Years,Years_in_Current_Job,Delinquency,Income_Profile,SENTIMENT_SCORE
0,0.470195,0.912798,0.188360,-1.390131,-0.787119,3,-1.029514,1,0,1,...,1,0,4,0.198556,-0.058821,2,0.553344,1,0,2
1,0.740538,-1.270699,1.044588,-0.613757,-0.045594,3,-0.947514,1,0,0,...,2,0,0,1.252893,-0.472278,1,-1.629688,1,1,1
2,-1.489795,0.127785,0.047786,0.248879,-0.095029,3,1.728081,1,0,1,...,2,1,1,-1.614325,0.285726,1,0.735263,0,2,1
3,-0.138078,0.176391,1.141145,0.421407,0.597061,2,-1.152513,1,1,1,...,0,1,4,0.168565,1.181549,1,1.644860,0,2,1
4,-0.949108,-0.265368,-1.505382,-1.390131,-0.144464,1,0.119210,0,1,0,...,1,1,0,0.220941,-1.299191,2,-0.356253,1,2,1


In [229]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, \
    classification_report

# Assuming `customer_data` is the dataset and "Delinquency_Status" is the target
X = customer_data_merged.drop("Delinquency", axis=1)
y = customer_data_merged["Delinquency"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Dictionary to store model evaluation results
results = {}
X_train.shape, y_train.shape, X_test.shape, y_test.shape


# Function to train, predict, and evaluate models
def evaluate_model(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None

    # Store results in a dictionary
    results[model_name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "ROC AUC": roc_auc
    }
    print(f"\n--- {model_name} ---")
    print(classification_report(y_test, y_pred))
    if roc_auc:
        print(f"ROC AUC: {roc_auc:.4f}")


# Initialize models
rf_model = RandomForestClassifier(random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
lgbm_model = LGBMClassifier(random_state=42)
# Train and evaluate Random Forest
evaluate_model(rf_model, "Random Forest")

# Train and evaluate XGBoost
evaluate_model(xgb_model, "XGBoost")

# Train and evaluate LightGBM
evaluate_model(lgbm_model, "LightGBM")

# Display evaluation results
pd.DataFrame(results).T
# saving the model with highest scores
import pickle

# Save the LightGBM model
with open("lightgbm_model.pkl", "wb") as file:
    pickle.dump(lgbm_model, file)
import numpy as np
import pandas as pd





--- Random Forest ---
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      2219
           1       0.96      0.84      0.90       781

    accuracy                           0.95      3000
   macro avg       0.95      0.91      0.93      3000
weighted avg       0.95      0.95      0.95      3000

ROC AUC: 0.9916


C:\Users\NBaggi\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [00:35:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



--- XGBoost ---
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2219
           1       0.95      0.94      0.95       781

    accuracy                           0.97      3000
   macro avg       0.97      0.96      0.96      3000
weighted avg       0.97      0.97      0.97      3000

ROC AUC: 0.9971
[LightGBM] [Info] Number of positive: 1848, number of negative: 5152
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001443 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1847
[LightGBM] [Info] Number of data points in the train set: 7000, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.264000 -> initscore=-1.025281
[LightGBM] [Info] Start training from score -1.025281

--- LightGBM ---
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2219
           1       0.9

,Credit_Score,Annual_Income,Loan_Amount,LTV_Ratio,Credit_Utilization_Ratio,Missed_Payments_Past_Year,Avg_Monthly_Balance,Employment_Status,Recent_Large_Transactions,Credit_History,...,Credit_Card_Balance,Industry,Age,Education_Level,Marital_Status,Number_of_Dependents,Total_Debt,Debt_to_Income_Ratio,Past_Delinquencies_2_5_Years,Years_in_Current_Job
0,730,88595,52848,0.59,0.6,1,8670,1,0,0,...,42343,Sports,32,Bachelor’s,Married,3,5521,0.59,1,14


In [238]:
# Generate an unseen sample
unseen_sample = {
    "Credit_Score": np.random.randint(700, 751),
    "Annual_Income": np.random.randint(40000, 120001),
    "Loan_Amount": np.random.randint(10000, 100001),
    "LTV_Ratio": np.round(np.random.uniform(0.5, 0.9), 2),
    "Credit_Utilization_Ratio": np.round(np.random.uniform(0.1, 0.8), 2),
    "Missed_Payments_Past_Year": np.random.randint(0, 4),
    "Avg_Monthly_Balance": np.random.randint(500, 10001),
    "Employment_Status": np.random.choice([0, 1], p=[0.1, 0.9]),
    "Recent_Large_Transactions": np.random.choice([0, 1], p=[0.85, 0.15]),
    "Credit_History": np.random.choice([0, 1], p=[0.6, 0.4]),
    "Savings_Account_Balance": np.random.randint(0, 100001),
    "Credit_Card_Balance": np.random.randint(0, 100001),
    "Industry": np.random.choice(['Technology', 'Sports','Finance']),
    "Age": np.random.randint(21, 65),
    "Education_Level": np.random.choice(['High School', 'Bachelor’s', 'Master’s', 'PhD']),
    "Marital_Status": np.random.choice(['Single', 'Married', 'Divorced']),
    "Number_of_Dependents": np.random.randint(0, 5),
    "Total_Debt": np.random.randint(5000, 150001),
    "Debt_to_Income_Ratio": np.round(np.random.uniform(0.1, 0.6), 2),
    "Past_Delinquencies_2_5_Years": np.random.randint(0, 3),
    "Years_in_Current_Job": np.random.randint(1, 20),
}
# Print the unseen sample
unseen_sample_df = pd.DataFrame([unseen_sample])
unseen_sample_df

,Credit_Score,Annual_Income,Loan_Amount,LTV_Ratio,Credit_Utilization_Ratio,Missed_Payments_Past_Year,Avg_Monthly_Balance,Employment_Status,Recent_Large_Transactions,Credit_History,...,Credit_Card_Balance,Industry,Age,Education_Level,Marital_Status,Number_of_Dependents,Total_Debt,Debt_to_Income_Ratio,Past_Delinquencies_2_5_Years,Years_in_Current_Job
0,738,110398,31387,0.79,0.22,2,4504,1,0,1,...,94459,Finance,41,Bachelor’s,Married,4,104557,0.36,0,4


In [239]:
#Cust data 
def get_income_profile(Annual_Income):
    if Annual_Income < 60000:
        return 'Low'
    elif 60000 <= Annual_Income <= 90000:
        return 'Medium'
    else:
        return 'High'
# Apply the function to create a new column 'Income_profile'
unseen_sample_df['Income_Profile'] = unseen_sample_df['Annual_Income'].apply(get_income_profile)
unseen_sample_df

,Credit_Score,Annual_Income,Loan_Amount,LTV_Ratio,Credit_Utilization_Ratio,Missed_Payments_Past_Year,Avg_Monthly_Balance,Employment_Status,Recent_Large_Transactions,Credit_History,...,Industry,Age,Education_Level,Marital_Status,Number_of_Dependents,Total_Debt,Debt_to_Income_Ratio,Past_Delinquencies_2_5_Years,Years_in_Current_Job,Income_Profile
0,738,110398,31387,0.79,0.22,2,4504,1,0,1,...,Finance,41,Bachelor’s,Married,4,104557,0.36,0,4,High


In [240]:
# Join the DataFrames on 'Industry' and 'Income_profile'/'Incomeprofile'
unseen_sample_merged_df = pd.merge(unseen_sample_df, tweet_data_df, how='inner', left_on=['Industry', 'Income_Profile'], right_on=['BUSINESS_SEGMENT', 'INCOME_PROFILE'])

# Drop the redundant 'Incomeprofile' and 'business_segment' columns from the merged DataFrame
unseen_sample_merged_df = unseen_sample_merged_df.drop(columns=['INCOME_PROFILE', 'BUSINESS_SEGMENT'])
unseen_sample_merged_df

,Credit_Score,Annual_Income,Loan_Amount,LTV_Ratio,Credit_Utilization_Ratio,Missed_Payments_Past_Year,Avg_Monthly_Balance,Employment_Status,Recent_Large_Transactions,Credit_History,...,Age,Education_Level,Marital_Status,Number_of_Dependents,Total_Debt,Debt_to_Income_Ratio,Past_Delinquencies_2_5_Years,Years_in_Current_Job,Income_Profile,SENTIMENT_SCORE
0,738,110398,31387,0.79,0.22,2,4504,1,0,1,...,41,Bachelor’s,Married,4,104557,0.36,0,4,High,Neutral


In [241]:
import joblib

# After fitting LabelEncoder and StandardScaler on training data
# Assuming 'le' is your LabelEncoder and 'scaler' is your StandardScaler

# Save the fitted LabelEncoder and StandardScaler to disk
joblib.dump(le_industry, 'label_encoder_industry.pkl')
joblib.dump(le_education, 'label_encoder_education.pkl')
joblib.dump(le_marital, 'label_encoder_marital.pkl') 
joblib.dump(le_SENTIMENT_SCORE, 'label_encoder_sentiment_score.pkl') 
joblib.dump(le_income_profile, 'label_encoder_income_profile.pkl') 
joblib.dump(scaler, 'scaler.pkl')
unseen_sample
# Later, when you want to preprocess new data, load the saved transformers
# Load the saved LabelEncoder and StandardScaler
loaded_le_industry = joblib.load('label_encoder_industry.pkl')
loaded_le_education = joblib.load('label_encoder_education.pkl')
loaded_le_marital = joblib.load('label_encoder_marital.pkl')
loaded_le_SENTIMENT_SCORE = joblib.load('label_encoder_sentiment_score.pkl')
loaded_le_income_profile = joblib.load('label_encoder_income_profile.pkl')
loaded_scaler = joblib.load('scaler.pkl')

# Now, you can use these loaded transformers on the new unseen data
#unseen_sample_df = pd.DataFrame([unseen_sample])
# Preprocess categorical variables using the loaded LabelEncoder
unseen_sample_merged_df['Industry'] = loaded_le_industry.transform(unseen_sample_merged_df['Industry'])
unseen_sample_merged_df['Education_Level'] = loaded_le_education.transform(unseen_sample_merged_df['Education_Level'])
unseen_sample_merged_df['Marital_Status'] = loaded_le_marital.transform(unseen_sample_merged_df['Marital_Status'])
unseen_sample_merged_df['SENTIMENT_SCORE'] = loaded_le_SENTIMENT_SCORE.transform(unseen_sample_merged_df['SENTIMENT_SCORE'])
unseen_sample_merged_df['Income_Profile'] = loaded_le_income_profile.transform(unseen_sample_merged_df['Income_Profile'])
# Scale numerical features using the loaded StandardScaler
unseen_sample_merged_df[numerical_features] = loaded_scaler.transform(unseen_sample_merged_df[numerical_features])

# Now your data is ready for prediction
print(unseen_sample_merged_df)



   Credit_Score  Annual_Income  Loan_Amount  LTV_Ratio  \
0       0.87571       1.321635    -0.928054   0.766461   

   Credit_Utilization_Ratio  Missed_Payments_Past_Year  Avg_Monthly_Balance  \
0                 -1.133164                          2            -0.265162   

   Employment_Status  Recent_Large_Transactions  Credit_History  ...  \
0                  1                          0               1  ...   

        Age  Education_Level  Marital_Status  Number_of_Dependents  \
0 -0.122436                0               1                     4   

   Total_Debt  Debt_to_Income_Ratio  Past_Delinquencies_2_5_Years  \
0    0.643991              0.078998                             0   

   Years_in_Current_Job  Income_Profile  SENTIMENT_SCORE  
0              -1.08393               0                1  

[1 rows x 23 columns]


In [242]:
# Load the LightGBM model
with open("lightgbm_model.pkl", "rb") as file:
    lgbm_model = pickle.load(file)
lgbm_pred = lgbm_model.predict(unseen_sample_merged_df)
print(lgbm_pred)
lgbm_prob = lgbm_model.predict_proba(unseen_sample_merged_df)
print(lgbm_prob)
#Check if our logic and output match or not
test = unseen_sample_merged_df.apply(calculate_delinquency_risk, axis=1)
test


[0]
[[9.99178454e-01 8.21546233e-04]]


0    0
dtype: int64

In [244]:
#testing model for delinquency value 1 which is in Input data row 8
row_dict = customer_data_merged.iloc[[8]].to_dict('records')[0]
row_dict.pop('Delinquency', None)
row_dict_df = pd.DataFrame([row_dict])
lgbm_pred = lgbm_model.predict(row_dict_df)
print(lgbm_pred)
lgbm_prob = lgbm_model.predict_proba(row_dict_df)
print(lgbm_prob)

[0]
[[0.98045433 0.01954567]]
